<a href="https://colab.research.google.com/github/eveapp/data_science/blob/Ketki/Topic%20Classification%20F20/Topic%20Modeling/Reddit_Text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we clean and preprocess our reddit data.

For cleaning we do the following to the values of 'body' column:
  - Remove punctuations
  - Convert to lowercase
  - Remove Word Contractions (e.g. I've to I have).
  - Remove Stop Words (e.g. has, at, etc)

For Text Preprocessing we do the following to the values of 'cleaned_body' column:
 - Tokenize the text
 - Tag Parts of speech using NLTK standard POS tagging
 - Convert std NLTK POS tagging to WordNet POS tagging format. 
 - Lemmatize the Wordnet POS tagged data

 We save the cleaned and Preprocessed data into a csv for further EDA


In [ ]:
pip install contractions

     |████████████████████████████████| 245kB 5.2MB/s 
     |████████████████████████████████| 317kB 13.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=0719335824af477a1a6fd46b1ddb4bce2ad132ce0c352f7b4ef71edac771c7f3
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import contractions
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_old=pd.read_csv('EAFC_Labelled.csv')
df_old.head()

,Unnamed: 0,score,id,url,comms_num,title,body,timestamp,clean_body,clean_body_str,tokenized,pos_tags,wordnet_pos,lemmatized,label,Physical Symptoms,Sentiment
0,NaN,88,d3mxw2,https://www.reddit.com/r/cancer/comments/d3mxw...,32,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",2019-09-13 17:55:58,"['sorry', 'title', 'wish', 'bell', 'ringing', ...",sorry title wish bell ringing party music cele...,"['sorry', 'title', 'wish', 'bell', 'ringing', ...","[('sorry', 'JJ'), ('title', 'NN'), ('wish', 'J...","[('sorry', 'a'), ('title', 'n'), ('wish', 'a')...","['sorry', 'title', 'wish', 'bell', 'ring', 'pa...",6.0,NaN,NaN
1,NaN,7,hesliu,https://www.reddit.com/r/CancerFamilySupport/c...,4,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,2020-06-24 10:35:34,"['dad', 'diagnosed', 'stage', 'bone', 'cancer'...",dad diagnosed stage bone cancer january made d...,"['dad', 'diagnosed', 'stage', 'bone', 'cancer'...","[('dad', 'NN'), ('diagnosed', 'VBD'), ('stage'...","[('dad', 'n'), ('diagnosed', 'v'), ('stage', '...","['dad', 'diagnose', 'stage', 'bone', 'cancer',...",7.0,NaN,NaN
2,NaN,76,50p82p,https://www.reddit.com/r/cancer/comments/50p82...,17,last chemo was today :),just finished my last of 12 abvd infusions. I ...,2016-09-02 4:01:24,"['finished', 'last', 'abvd', 'infusions', 'fee...",finished last abvd infusions feel terrible kno...,"['finished', 'last', 'abvd', 'infusions', 'fee...","[('finished', 'VBN'), ('last', 'JJ'), ('abvd',...","[('finished', 'v'), ('last', 'a'), ('abvd', 'a...","['finish', 'last', 'abvd', 'infusion', 'feel',...",9.0,NaN,NaN
3,NaN,2,dg7uwq,https://www.reddit.com/r/CancerCaregivers/comm...,2,ER phobia?,Have any of your loved ones checked out of the...,2019-10-11 9:22:41,"['loved', 'ones', 'checked', 'hospital', 'ama'...",loved ones checked hospital ama medical advice...,"['loved', 'ones', 'checked', 'hospital', 'ama'...","[('loved', 'VBN'), ('ones', 'NNS'), ('checked'...","[('loved', 'v'), ('ones', 'n'), ('checked', 'v...","['love', 'one', 'check', 'hospital', 'ama', 'm...",7.0,NaN,NaN
4,NaN,4,bry6ro,https://www.reddit.com/r/CancerCaregivers/comm...,10,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,2019-05-23 11:16:30,"['point', 'it is', 'funny', 'I am', 'one', 'th...",point it is funny I am one throwing temper tan...,"['point', 'it', 'is', 'funny', 'I', 'am', 'one...","[('point', 'NN'), ('it', 'PRP'), ('is', 'VBZ')...","[('point', 'n'), ('it', 'n'), ('is', 'v'), ('f...","['point', 'it', 'be', 'funny', 'I', 'be', 'one...",1.0,NaN,NaN


In [ ]:
df = df_old[['title', 'body', 'label']].copy()

In [ ]:
df1 = pd.read_csv('RedditCancer.csv')
df1.head()

df2 = pd.read_csv('RedditCancerCaregivers.csv')
df2.head()

df3 = pd.read_csv('RedditCancerFamilySupport.csv')
df3.head()

df = pd.concat([df1,df2], axis=0)
df = pd.concat([df,df3], axis=0)


In [ ]:
df.to_csv('EAFC_Raw.csv')

In [ ]:
#(#Rows, #Columns)
df.shape

(480, 5)

In [ ]:
df=df.dropna()
df.head()

,title,body,label,clean_body,clean_body_str
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...


In [ ]:
#Rows that have values we could use
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 845
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   480 non-null    object 
 1   body    480 non-null    object 
 2   label   480 non-null    float64
dtypes: float64(1), object(2)
memory usage: 15.0+ KB


In [ ]:
#Detect Language of each 
# pretrained_model = "lid.176.bin" 
# model = fasttext.load_model(pretrained_model)
# langs = []
# for sent in rws['rating_description_str']:
#     lang = model.predict(sent)[0]
#     langs.append(str(lang)[11:13])
# rws['langs'] = langs

In [ ]:
#Make everything lowercase
df['clean_body'] =  df["title"].str.lower()+" "+ df["body"].str.lower() 

In [ ]:
def remove_punctuations_numbers(text):
    punc = string.punctuation + '0123456789'
    for punctuation in punc:
        text = text.replace(punctuation, '')
    return text

In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
#remove punctuation 

df["clean_body"] = df["clean_body"].astype(str)
df['clean_body'] = df['clean_body'].apply(remove_punctuations_numbers)
df.head()

,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,fuck that bell sorry for the title but i wish ...
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,father suffering from end stage aggression won...
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,last chemo was today just finished my last of...
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,er phobia have any of your loved ones checked ...
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,minutes into his appointment another scene ma...


In [ ]:
#Remove Word contractions
df['clean_body'] = df['clean_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df.head()

,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, that, bell, sorry, for, the, title, but..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, from, end, stage, aggressi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, was, today, just, finished, my, ..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, have, any, of, your, loved, ones,..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, into, his, appointment, another, sce..."


In [ ]:
#Remove Stop Words
stop_words = set(stopwords.words('english'))
print(len(stop_words))
stop_words.add('i')
stop_words.add('even')
stop_words.add('still')
stop_words.add('ever')
stop_words.add('really')
stop_words.add('seem')
stop_words.add('almost')
stop_words.add('go')
print(len(stop_words))
df['clean_body'] = df['clean_body'].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

179
186


,title,body,label,clean_body
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p..."


In [ ]:
#Clean body Str:
df['clean_body_str'] = [' '.join(map(str, l)) for l in df['clean_body']]
df.head()

,title,body,label,clean_body,clean_body_str
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...


In [ ]:
#Tokenize
df['tokenized'] = df['clean_body_str'].apply(word_tokenize)
df.head()

,title,body,label,clean_body,clean_body_str,tokenized
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...,"[father, suffering, end, stage, aggression, wi..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...,"[last, chemo, today, finished, last, abvd, inf..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...,"[er, phobia, loved, ones, checked, hospital, a..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...,"[minutes, appointment, another, scene, made, p..."


In [ ]:
#Tagging Parts of Speech in the tokenized 
df['pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag)
df.head()

,title,body,label,clean_body,clean_body_str,tokenized,pos_tags
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...,"[father, suffering, end, stage, aggression, wi...","[(father, RB), (suffering, VBG), (end, NN), (s..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...,"[last, chemo, today, finished, last, abvd, inf...","[(last, JJ), (chemo, NN), (today, NN), (finish..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...,"[er, phobia, loved, ones, checked, hospital, a...","[(er, NN), (phobia, NN), (loved, VBD), (ones, ..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...,"[minutes, appointment, another, scene, made, p...","[(minutes, NNS), (appointment, VBP), (another,..."


In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
df.head()


,title,body,label,clean_body,clean_body_str,tokenized,pos_tags,wordnet_pos
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ...","[(fuck, n), (bell, n), (sorry, a), (title, n),..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...,"[father, suffering, end, stage, aggression, wi...","[(father, RB), (suffering, VBG), (end, NN), (s...","[(father, r), (suffering, v), (end, n), (stage..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...,"[last, chemo, today, finished, last, abvd, inf...","[(last, JJ), (chemo, NN), (today, NN), (finish...","[(last, a), (chemo, n), (today, n), (finished,..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...,"[er, phobia, loved, ones, checked, hospital, a...","[(er, NN), (phobia, NN), (loved, VBD), (ones, ...","[(er, n), (phobia, n), (loved, v), (ones, n), ..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...,"[minutes, appointment, another, scene, made, p...","[(minutes, NNS), (appointment, VBP), (another,...","[(minutes, n), (appointment, v), (another, n),..."


In [ ]:
#Lemmatization
wnl = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
df.head()

,title,body,label,clean_body,clean_body_str,tokenized,pos_tags,wordnet_pos,lemmatized
0,Fuck that Bell!,"Sorry for the title, but I wish the bell ringi...",6.0,"[fuck, bell, sorry, title, wish, bell, ringing...",fuck bell sorry title wish bell ringing party ...,"[fuck, bell, sorry, title, wish, bell, ringing...","[(fuck, NN), (bell, NN), (sorry, JJ), (title, ...","[(fuck, n), (bell, n), (sorry, a), (title, n),...","[fuck, bell, sorry, title, wish, bell, ring, p..."
1,Father suffering from end stage aggression. Wo...,My dad was diagnosed with stage 4 bone cancer ...,7.0,"[father, suffering, end, stage, aggression, wi...",father suffering end stage aggression will not...,"[father, suffering, end, stage, aggression, wi...","[(father, RB), (suffering, VBG), (end, NN), (s...","[(father, r), (suffering, v), (end, n), (stage...","[father, suffer, end, stage, aggression, will,..."
2,last chemo was today :),just finished my last of 12 abvd infusions. I ...,9.0,"[last, chemo, today, finished, last, abvd, inf...",last chemo today finished last abvd infusions ...,"[last, chemo, today, finished, last, abvd, inf...","[(last, JJ), (chemo, NN), (today, NN), (finish...","[(last, a), (chemo, n), (today, n), (finished,...","[last, chemo, today, finish, last, abvd, infus..."
3,ER phobia?,Have any of your loved ones checked out of the...,7.0,"[er, phobia, loved, ones, checked, hospital, a...",er phobia loved ones checked hospital ama medi...,"[er, phobia, loved, ones, checked, hospital, a...","[(er, NN), (phobia, NN), (loved, VBD), (ones, ...","[(er, n), (phobia, n), (loved, v), (ones, n), ...","[er, phobia, love, one, check, hospital, ama, ..."
4,"8 minutes into his appointment, another scene ...",At this point it’s just funny to me. I’m not t...,1.0,"[minutes, appointment, another, scene, made, p...",minutes appointment another scene made plus ho...,"[minutes, appointment, another, scene, made, p...","[(minutes, NNS), (appointment, VBP), (another,...","[(minutes, n), (appointment, v), (another, n),...","[minute, appointment, another, scene, make, pl..."


In [ ]:
df.to_pickle("Reddit_dataset_clean.pkl")

In [ ]:
df.to_csv('Reddit_dataset_clean_labelled.csv')